# Backends: t|ket> example

This example shows how to use `pytket` to execute quantum circuits on both simulators and real devices, and how to interpret the results. As t|ket> is designed to be platform-agnostic, we have unified the interfaces of different providers as much as possible into the `Backend` class for maximum portability of code. Currently, the following are supported:

* ProjectQ simulator
* Aer simulators (statevector, QASM, and unitary)
* IBMQ devices
* Rigetti QCS devices
* Rigetti QVM (for device simulation or statevector)

Our Rigetti example introduces the Rigetti backends, so we will focus on the others here.

To get started, we must install core pytket and the subpackages required to interface with the two providers:

```
pip install pytket
pip install pytket_qiskit
pip install pytket_projectq
```

First, import the backends that we will be demonstrating.

In [1]:
from pytket.backends.ibm import AerStateBackend, AerBackend, AerUnitaryBackend, IBMQBackend
from pytket.backends.projectq import ProjectQBackend

We are also going to be making a circuit to run on these backends, so import the `Circuit` class.

In [2]:
from pytket import Circuit

Below we generate a circuit which will produce a Bell state, assuming the qubits are all initialised in the |0> state:

In [3]:
circ = Circuit(2)
circ.H(0)
circ.CX(0,1)

<tket::Circuit qubits=2, gates=2>

As a sanity check, we will use the `AerStateBackend` to verify that `circ` does actually produce a Bell state.

Calling `get_state` on a backend will execute the circuit and return a `numpy` array corresponding to the statevector. This style of usage is used consistently in the `pytket` backends.

In [4]:
aer_state_b = AerStateBackend()
statevector = aer_state_b.get_state(circ)
print(statevector)

[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


As we can see, the output state vector $\lvert \psi_{\mathrm{circ}}\rangle$ is $(\lvert00\rangle + \lvert11\rangle)/\sqrt2$.

This is a symmetric state. For non-symmetric states, we default to an ILO-BE format (increasing lexicographic order of (qu)bit ids, big-endian), but an alternative convention can be specified when retrieving results from backends. See the docs for the `BasisOrder` enum for more information.

A lesser-used simulator available through Qiskit Aer is their unitary simulator. This will be somewhat more expensive to run, but returns the full unitary matrix for the provided circuit. This is useful in the design of small subcircuits that will be used multiple times within other larger circuits - statevector simulators will only test that they act correctly on the $\lvert 0 \rangle^{\otimes n}$ state, which is not enough to guarantee the circuit's correctness.

The `AerUnitaryBackend` provides a convenient access point for this simulator for use with `pytket` circuits.

In [5]:
aer_unitary_b = AerUnitaryBackend()
print(aer_unitary_b.get_unitary(circ))

[[ 0.70710678+0.j  0.        +0.j  0.70710678+0.j  0.        +0.j]
 [ 0.        +0.j  0.70710678+0.j  0.        +0.j  0.70710678+0.j]
 [ 0.        +0.j  0.70710678+0.j  0.        +0.j -0.70710678+0.j]
 [ 0.70710678+0.j  0.        +0.j -0.70710678+0.j  0.        +0.j]]


Now suppose we want to measure this Bell state to get some actual results out, so let's append some `Measure` gates to the circuit. The `Circuit` class has the `measure_all` utility function which appends `Measure` gates on every qubit. All of these results will be written to the default classical register ('c'). This function will automatically add the classical bits to the circuit if they are not already there.

In [6]:
circ.measure_all()

<tket::Circuit qubits=2, gates=4>

We can get some shots out from the `AerBackend`, which is an interface to the Qiskit Aer QASM simulator. Suppose we would like to get 10 shots out. We can seed the simulator's random-number generator in order to make the results reproducible.

In [7]:
aer_b = AerBackend()
shots = aer_b.get_shots(circuit=circ, n_shots=10, seed=1)
print(shots)

[[1 1]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [1 1]]


Shot tables are just numpy arrays where each row gives the final readout for each circuit run, and each column represents one of the classical bits in the circuit (ordered right-to-left to fit the little-endian convention, so the row `[1 0]` means bit 0 had value 0, and bit 1 had value 1).

In this case there is a 40/60 split between $00$ and $11$ results. If we change the seed, or remove it, we will get varying results according to the pseudo-random number generation internal to Qiskit's QASM simulator.

What happens if we simulate some noise in our imagined device, using the Qiskit Aer noise model?

To investigate this, we will require an import from Qiskit. For more information about noise modelling using Qiskit Aer, see the [Qiskit device noise simulation](https://qiskit.org/documentation/aer/device_noise_simulation.html) documentation.

In [8]:
from qiskit.providers.aer.noise import NoiseModel
my_noise_model = NoiseModel()
readout_error = 0.2
my_noise_model.add_all_qubit_readout_error([[1-readout_error, readout_error], [readout_error, 1-readout_error]])

This simple noise model gives a 20% chance that, upon measurement, a qubit that would otherwise have been measured as $0$ would instead be measured as $1$, and vice versa. Let's see what our shot table looks like with this model:

In [9]:
noisy_aer_b = AerBackend(my_noise_model)
noisy_shots = noisy_aer_b.get_shots(circuit=circ, n_shots=10, seed=1)
print(noisy_shots)

[[1 1]
 [0 0]
 [1 1]
 [1 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [0 1]]


We now have some spurious $01$ and $10$ measurements, which could never happen when measuring a Bell state on a noiseless device. 

The `AerBackend` class can accept any Qiskit noise model.

Suppose that we don't need the full shot table, but just want a summary of the results. The most common summary is the counts map, mapping the readout state to the number of times it was observed.

Some backends have direct support for this via the `get_counts` method, returning the counts map immediately. If we already have the shot table, `pytket` provides a quick utility function to summarise the table.

In [10]:
from pytket.utils import counts_from_shot_table
print(counts_from_shot_table(noisy_shots))

{(0, 0): 5, (0, 1): 1, (1, 0): 1, (1, 1): 3}


The last simulator we will demonstrate is the `ProjectQBackend`. ProjectQ offers fast simulation of quantum circuits with built-in support for fast expectation values from operators. The `ProjectQBackend` exposes this functionality to take in OpenFermion `QubitOperator` instances.

Note: ProjectQ can also produce shots in the style of `AerBackend`, using the same method, but cannot accept a noise model.

Let's create a `QubitOperator` object and a new circuit:

In [11]:
from openfermion import QubitOperator

hamiltonian = 0.5 * QubitOperator('X0 X2') + 0.3 * QubitOperator('Z0')

In [12]:
circ2 = Circuit(3)
circ2.Y(0)
circ2.H(1)
circ2.Rx(0.3, 2)

<tket::Circuit qubits=3, gates=3>

Now we can create a `ProjectQBackend` instance and feed it our circuit and `QubitOperator`:

In [13]:
projectq_b = ProjectQBackend()
expectation = projectq_b.get_operator_expectation_value(circ2, hamiltonian)
print(expectation)

-0.2999999999999999


The last leg of this tour includes running a pytket circuit on an actual quantum computer. To do this, you will need an IBM quantum experience account and have your credentials stored on your computer. See https://quantum-computing.ibm.com to make an account and view available devices and their specs.

Physical devices have much stronger constraints on the form of admissible circuits than simulators. They tend to support a minimal gate set, have restricted connectivity between qubits for two-qubit gates, and can have limited support for classical control flow or conditional gates. This is where we can invoke the t|ket> compiler passes to transform our desired circuit into one that is suitable for the backend.

Let's create an `IBMQBackend` for the `ibmq_essex` device and check if our circuit is valid to be run.

In [14]:
ibmq_b = IBMQBackend('ibmq_essex')
ibmq_b.valid_circuit(circ)

False

It looks like we need to compile this circuit to be compatible with the device. To simplify this procedure, we provide a minimal compilation pass designed for each backend (the `default_compilation_pass` property) which will guarantee compatibility with the device. These may still fail if the input circuit has too many qubits or unsupported usage of conditional gates. The default passes will typically do very little in the way of circuit optimisation, but they can be easily composed with any of t|ket>'s other optimisation passes for better performance.

For convenience, we also wrap up this pass into the `compile_circuit` method if you just want to compile a single circuit.

In [15]:
ibmq_b.compile_circuit(circ)
ibmq_b.valid_circuit(circ)

True

We are now good to run this circuit on the device.

In [16]:
quantum_shots = ibmq_b.get_shots(circuit=circ, n_shots=10)
print(quantum_shots)

Job Status: job has successfully run
[[1 1]
 [1 1]
 [1 1]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 0]
 [0 0]
 [1 1]]


These are from an actual device, so it's impossible to perfectly predict what the results will be. However, because of the problem of noise, it would be unsurprising to find a few $01$ or $10$ results in the table. The circuit is very short, so it should be fairly close to the ideal result.

The devices available through the IBM Q Experience serve jobs one at a time from their respective queues, so a large amount of experiment time can be taken up by waiting for your jobs to reach the front of the queue. `pytket` allows circuits to be submitted to any backend in a single batch using the `process_circuits` method. For the `IBMQBackend`, this will collate the circuits into as few jobs as possible which will all be sent off into the queue for the device. Any results will be cached in the backend object until they are retrieved. The `get_(shots/counts/state/unitary)` method will return any cached results for the circuit if they are available (blocking the process if the evaluation is in progress), and will otherwise submit the circuit for evaluation.

In [17]:
circuits = []
for i in range(5):
    c = Circuit(2)
    c.Rx(0.2*i,0).CX(0,1)
    c.measure_all()
    ibmq_b.compile_circuit(c)
    circuits.append(c)
ibmq_b.process_circuits(circuits, n_shots=100)

We can now retrieve the results and process them. As we measured each circuit in the $Z$-basis, we can obtain the expectation value for the $ZZ$ operator immediately from these measurement results. We can calculate this using the `expectation_value_from_shots` utility method in `pytket`.

In [18]:
from pytket.utils import expectation_from_shots
for c in circuits:
    shots = ibmq_b.get_shots(c)
    exp_val = expectation_from_shots(shots)
    print(exp_val)

Job Status: job has successfully run
-0.8400000000000001
Job Status: job has successfully run
-0.8600000000000001
Job Status: job has successfully run
-0.78
Job Status: job has successfully run
-0.72
Job Status: job has successfully run
-0.74


The backends in `pytket` are designed to be as similar to one another as possible. The example above using physical devices can be run entirely on a simulator by swapping out the `IBMQBackend` constructor for any other backend supporting shot outputs (`AerBackend`, `ProjectQBackend`, `ForestBackend`, or passing it the name of a different device) and it is simple to convert between handling shot tables, counts maps, and statevectors.

For more information on backends and other `pytket` features, read our [documentation](https://cqcl.github.io/pytket) or see the other examples on our [GitHub repo](https://github.com/CQCL/pytket).